In [1]:
import glob
import numpy             as np
import pandas            as pd
import tables            as tb
import matplotlib.pyplot as plt

from invisible_cities.io.pmaps_io import load_pmaps_as_df

to_df = pd.DataFrame.from_records

In [2]:
dtype = "pmaps"
run_number = 7255

icversion_datadir   = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/v0.9.9/"
testprod_datadir   = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/IC-testprod/"
testprod_p_datadir = f"/Users/gonzalo/Documents/NEXT/ireneProblem/DATA/{run_number}/IC-testprod-p/"

icversion_files  = glob.glob(icversion_datadir  + f"*")
testprod_files   = glob.glob(testprod_datadir   + f"*")
testprod_p_files = glob.glob(testprod_p_datadir + f"*")

icversion_files .sort()
testprod_files  .sort()
testprod_p_files.sort()

In [3]:
_, S2, S2Si, _, _ = load_pmaps_as_df(icversion_files[0])

In [ ]:
# for each event
# for each S2 (peak)

# total charge
# number of SiPMs touched

# charge per slide

In [4]:
S2 = S2Si[(S2Si["event"]==1)&(S2Si["peak"]==0)]

In [19]:
def Pmaps_SiSignal_info(pmapdir, npmaps, outfilename):
    
    Q, n_si = [], []
    Q_per_slide = []
    
    evts = []
    
    FILES = glob.glob(pmapdir + "/*")
    FILES.sort()
    FILES = FILES[:npmaps]
    
    for pmap in FILES:
        
        _, S2pmap, S2Sipmap, _, _ = load_pmaps_as_df(pmap)
        
        EVENTS = np.unique(S2pmap["event"].values)
        
        #loop in events
        for ev in EVENTS:
            S2   = S2pmap  [S2pmap  ["event"]==ev]
            S2Si = S2Sipmap[S2Sipmap["event"]==ev]
            
            #loop in peaks
            PEAKS = np.unique(S2Si["peak"])
            for pk in PEAKS:
                S2Si_pk = S2Si [ S2Si["peak"] == pk ]
                
                #print(ev, pk, len(S2Si_pk))
                
                #total change and number of sipms
                Q   .append(        np.sum(S2Si_pk["ene"]) )
                n_si.append( len(np.unique(S2Si_pk["nsipm"])) )
                evts.append( ev )
                
                nslides = len(S2[S2["peak"]==pk])
                sipms    = np.unique(S2Si_pk["nsipm"].values)
                
                b = S2Si_pk.index[0]
                #charge per slide
                for sl in range(0, int(nslides)-1):
                    q = []
                    for si in range(0, len(sipms)):
                        sl_si = S2Si_pk[S2Si_pk.index == b + sl + nslides*si]
                        q.append( sl_si["ene"].values )
                        
                    Q_per_slide.append( np.sum(q) )
                          
    si_df = pd.DataFrame({
                'event' : evts,
                'charge': Q   ,
                'touch' : n_si})
                    
    qsl_df = pd.DataFrame({
                'charge_per_sl': Q_per_slide})
    
    si_df .to_hdf(outfilename, key='Q')
    qsl_df.to_hdf(outfilename, key='sl')

In [20]:
pmapdir = icversion_datadir
npmaps = 2
outfilename = "./summary_sisignal_icversion.h5"

Pmaps_SiSignal_info(pmapdir, npmaps, outfilename)

In [21]:
pd.read_hdf(outfilename, key="Q").head()

,event,charge,touch
0,1,588.712524,21
1,2,375.908447,18
2,3,507.163910,19
3,4,513.749817,17
4,5,673.763306,21


In [24]:
pd.read_hdf(outfilename, key="sl").head()

,charge_per_sl
0,1.521070
1,10.622007
2,22.775034
3,39.714458
4,88.548683
